# Deserialization attacks

## Ethical hacking 2022

## Alessandro Bruni

# References

- [Marshalling Pickles (Slides + Video)](http://frohoff.github.io/appseccali-marshalling-pickles/)
- [ysoserial: Deserialization attack framework for Java](https://github.com/frohoff/ysoserial)
- [ysoserial.net: .NET deserialization attack framework](https://github.com/pwntester/ysoserial.net)
- [Remote code execution on Microsoft Exchange](https://www.zerodayinitiative.com/blog/2020/2/24/cve-2020-0688-remote-code-execution-on-microsoft-exchange-server-through-fixed-cryptographic-keys)
- [Log4j vulnerability explained](https://www.youtube.com/watch?v=w2F67LbEtnk)
https://blog.tneitzel.eu/posts/01-attacking-java-rmi-via-ssrf/

# Intro

- serialization/deserialization is a widely-used concept in modern languages for storing objects (structured data)

In [12]:
# example in Python:
import pickle

class A:
    def __init__(self, x):
        print(x)
        self.x = x
    
    def __del__(self):
        print(self.x)

In [13]:
# Serialize the class
a = A("hello")
a_bytes = pickle.dumps(a)
del(a)

hello
hello


In [15]:
# Deserialize the class
print(a_bytes)
# a_bytes = a_bytes[:-5] + b'a' + a_bytes[-4:]
a = pickle.loads(a_bytes)
del(a)

b'\x80\x04\x95%\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x01A\x94\x93\x94)\x81\x94}\x94\x8c\x01x\x94\x8c\x05hello\x94sb.'
hella


## The problem?

Object integrity is lost once you pass it through some untrusted channel
- give it to the user to store some temporary data (think authentication cookies stored this way)
- send over the network
- store it in a database

## The solution?

- sanity checks required: e.g. cryptographically sign serialized objects to maintain integrity
- Java (e.g. RMI) does type checking: the receiving end deserializes the object, checks the type, if the type does not match then destroy the object

## Remote Code Execution with destructors

- RMI receives an object of a class present in your classpath
- constructs the object, checks the type (mismatch), destroys the object
- the destructor executes arbitrary code from the user
- if anywhere in your classpath you have an object that executes arbitrary code upon destruction, you have a path to RCE